In [72]:
import os
from dotenv import load_dotenv
# api_key = os.environ.get("OPENAI_API_KEY")
# os.environ['OPENAI_API_KEY'] = api_key
load_dotenv()

True

# Init LLM

In [14]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, model_kwargs={"seed" : 1337})

# Create a Prompt Template

In [15]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("{content}")

# Output Parser

In [16]:
from langchain.schema import StrOutputParser

str_output_parser = StrOutputParser()

# LLM

In [17]:
print(llm.input_schema.schema())

{'title': 'ChatOpenAIInput', 'anyOf': [{'type': 'string'}, {'$ref': '#/definitions/StringPromptValue'}, {'$ref': '#/definitions/ChatPromptValueConcrete'}, {'type': 'array', 'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'}, {'$ref': '#/definitions/HumanMessage'}, {'$ref': '#/definitions/ChatMessage'}, {'$ref': '#/definitions/SystemMessage'}, {'$ref': '#/definitions/FunctionMessage'}, {'$ref': '#/definitions/ToolMessage'}]}}], 'definitions': {'StringPromptValue': {'title': 'StringPromptValue', 'description': 'String prompt value.', 'type': 'object', 'properties': {'text': {'title': 'Text', 'type': 'string'}, 'type': {'title': 'Type', 'default': 'StringPromptValue', 'enum': ['StringPromptValue'], 'type': 'string'}}, 'required': ['text']}, 'AIMessage': {'title': 'AIMessage', 'description': 'A Message from an AI.', 'type': 'object', 'properties': {'content': {'title': 'Content', 'anyOf': [{'type': 'string'}, {'type': 'array', 'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}

In [18]:
print(llm.output_schema.schema())

{'title': 'ChatOpenAIOutput', 'anyOf': [{'$ref': '#/definitions/AIMessage'}, {'$ref': '#/definitions/HumanMessage'}, {'$ref': '#/definitions/ChatMessage'}, {'$ref': '#/definitions/SystemMessage'}, {'$ref': '#/definitions/FunctionMessage'}, {'$ref': '#/definitions/ToolMessage'}], 'definitions': {'AIMessage': {'title': 'AIMessage', 'description': 'A Message from an AI.', 'type': 'object', 'properties': {'content': {'title': 'Content', 'anyOf': [{'type': 'string'}, {'type': 'array', 'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]}, 'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'}, 'type': {'title': 'Type', 'default': 'ai', 'enum': ['ai'], 'type': 'string'}, 'example': {'title': 'Example', 'default': False, 'type': 'boolean'}}, 'required': ['content']}, 'HumanMessage': {'title': 'HumanMessage', 'description': 'A Message from a human.', 'type': 'object', 'properties': {'content': {'title': 'Content', 'anyOf': [{'type': 'string'}, {'type': 'array', 'items'

# StrOutputParser

In [19]:
print(str_output_parser.input_schema.schema())

{'title': 'StrOutputParserInput', 'anyOf': [{'type': 'string'}, {'$ref': '#/definitions/AIMessage'}, {'$ref': '#/definitions/HumanMessage'}, {'$ref': '#/definitions/ChatMessage'}, {'$ref': '#/definitions/SystemMessage'}, {'$ref': '#/definitions/FunctionMessage'}, {'$ref': '#/definitions/ToolMessage'}], 'definitions': {'AIMessage': {'title': 'AIMessage', 'description': 'A Message from an AI.', 'type': 'object', 'properties': {'content': {'title': 'Content', 'anyOf': [{'type': 'string'}, {'type': 'array', 'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]}, 'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'}, 'type': {'title': 'Type', 'default': 'ai', 'enum': ['ai'], 'type': 'string'}, 'example': {'title': 'Example', 'default': False, 'type': 'boolean'}}, 'required': ['content']}, 'HumanMessage': {'title': 'HumanMessage', 'description': 'A Message from a human.', 'type': 'object', 'properties': {'content': {'title': 'Content', 'anyOf': [{'type': 'string'}, {

In [20]:
print(str_output_parser.input_schema.schema())

{'title': 'StrOutputParserInput', 'anyOf': [{'type': 'string'}, {'$ref': '#/definitions/AIMessage'}, {'$ref': '#/definitions/HumanMessage'}, {'$ref': '#/definitions/ChatMessage'}, {'$ref': '#/definitions/SystemMessage'}, {'$ref': '#/definitions/FunctionMessage'}, {'$ref': '#/definitions/ToolMessage'}], 'definitions': {'AIMessage': {'title': 'AIMessage', 'description': 'A Message from an AI.', 'type': 'object', 'properties': {'content': {'title': 'Content', 'anyOf': [{'type': 'string'}, {'type': 'array', 'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]}, 'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'}, 'type': {'title': 'Type', 'default': 'ai', 'enum': ['ai'], 'type': 'string'}, 'example': {'title': 'Example', 'default': False, 'type': 'boolean'}}, 'required': ['content']}, 'HumanMessage': {'title': 'HumanMessage', 'description': 'A Message from a human.', 'type': 'object', 'properties': {'content': {'title': 'Content', 'anyOf': [{'type': 'string'}, {

In [42]:
chain = prompt | llm | str_output_parser

In [43]:
chain.invoke({"content" : "what is this week's challenge document about?"})

"I'm sorry, but as an AI language model, I don't have access to real-time information or specific documents. Therefore, I cannot provide you with details about this week's challenge document. If you have any other questions or need assistance with something else, feel free to ask!"

# Prompt Template

In [44]:
RAG_PROMPT_TEMPLATE = """\
Use the following context to answer the user's questions. If you don't know the answer, please respond with 'I don't know'.

Context:
{context}

Question:
{question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

In [45]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

# Embeddings and Vector Store

In [52]:
loader = TextLoader('./week_6_challenge_doc.txt')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 1000, chunk_overlap=200, model_name = "gpt-4-1106-preview")
texts  = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
store = Chroma.from_documents(texts,embeddings, collection_name="challenge_document")
store

# Setting up RAG chain

In [56]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

retriever = store.as_retriever()


entry_point_and_retriever = RunnableParallel(
    {
        "context" : retriever,
        "question" : RunnablePassthrough()
    }
)

rag_chain = entry_point_and_retriever | rag_prompt | llm | str_output_parser

In [57]:
rag_chain.invoke("what is this week's challenge document about?")

"The week's challenge document is about prompt engineering and understanding key performance indicators such as prompt ranking, prompt matching, and the ability to reuse previous knowledge."

# Elevated RAG with LangChain

In [87]:
import sys
!{sys.executable} -m pip install --upgrade --quiet  cohere

In [60]:
from langchain.retrievers import BM25Retriever

In [70]:
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 5

In [77]:
from langchain.embeddings import CohereEmbeddings

cohere_embeddings = CohereEmbeddings(model="embed-english-light-v3.0")

chroma_vectorstore = Chroma.from_documents(texts, cohere_embeddings)
chroma_retriever = chroma_vectorstore.as_retriever(search_kwargs={"k": 5})

In [81]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], weights=[0.75, 0.25]
)

In [109]:
from langchain.retrievers.document_compressors import CohereRerank
import cohere
cohere_api_key = os.environ['COHERE_API_KEY']
client_r = cohere.Client(cohere_api_key)
CohereRerank.update_forward_refs(client=client_r)
reranker = CohereRerank(top_n=5,)

NameError: name 'Client' is not defined

In [92]:
from langchain.retrievers import ContextualCompressionRetriever

rerank_retriever = ContextualCompressionRetriever(
    base_compressor=reranker, base_retriever=ensemble_retriever
)

NameError: name 'reranker' is not defined